### 6. Generování náhodných čísel a testování generátorů

#### Zadání:
Tento úkol bude poněkud kreativnější charakteru. Vaším úkolem je vytvořit vlastní generátor semínka do pseudonáhodných algoritmů. Jazyk Python umí sbírat přes ovladače hardwarových zařízení různá fyzická a fyzikální data. Můžete i sbírat data z historie prohlížeče, snímání pohybu myší, vyzvání uživatele zadat náhodné úhozy do klávesnice a jiná unikátní data uživatelů.

#### Vypracované řešení
1. Získávání dat z pohybu myši
2. Získávání dat z momentálních statistik HW komponent
3. Získávání dat z informací o počítači v síti
4. Získávání dat z informací o HW specifikací počítače
5. Získávání momentálního času v milisekundách

Na základě těchto bodů se generuje výsledný náhodný seed (semínko).

In [12]:
from time import time, time_ns
import hashlib
import random
import psutil
import socket
import platform

In [17]:
def get_entropy_mouse():
    entropy = ""
    start_time = time()
    while time() - start_time < 1:
        entropy += str(random.random())
    return hashlib.sha384(entropy.encode()).digest()


def get_entropy_system_performance():
    cpu_load = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent
    disk_io = (
        psutil.disk_io_counters().read_bytes + psutil.disk_io_counters().write_bytes
    )
    performance_data = f"{cpu_load},{memory_usage},{disk_io}"
    return hashlib.sha384(performance_data.encode()).digest()


def get_entropy_network_info():
    hostname = socket.gethostname()
    ip_address = socket.gethostbyname(hostname)
    network_data = f"{hostname},{ip_address}"
    return hashlib.sha384(network_data.encode()).digest()


def get_entropy_device_specifics():
    architecture = platform.architecture()
    machine = platform.machine()
    system = platform.system()
    device_data = f"{architecture},{machine},{system}"
    return hashlib.sha384(device_data.encode()).digest()


def get_time_in_ns() -> int:
    current_time_in_nanoseconds = time_ns()
    rounded_time_in_ns = int(round(current_time_in_nanoseconds))
    return rounded_time_in_ns


def get_seed():
    seed = bytearray()

    nanoseconds = get_time_in_ns()
    mouse = get_entropy_mouse()
    performance = get_entropy_system_performance()
    traffic = get_entropy_network_info()
    device = get_entropy_device_specifics()

    seed.extend(mouse)
    seed.extend(nanoseconds.to_bytes(16, "little"))
    seed.extend(performance)
    seed.extend(traffic)
    seed.extend(device)

    seed_hash = hashlib.sha384(seed).hexdigest()
    return seed_hash


seed = get_seed()
print(f"Seed: {seed}")

Seed: f049a2d81b0e436dc97501aa6a7ad4d1d8bb0d51b649210032911c41d1e3879ef1696a06bbe5615cd94c6d31a441b429
